In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

In [ ]:
# Load your dataset
df = pd.read_excel('../data/processed/exploratory_analysis_final.xlsx', sheet_name='Sheet1')

In [ ]:
# Interaction features
df['gpa_gre_interaction'] = df['undergrad_gpa'] * df['gre_avg']
df['gpa_acceptance_interaction'] = df['undergrad_gpa'] * df['acceptance_rate']

# Composite profile strength
df['profile_strength'] = df['gre_avg'] * df['gpa_percentile'] * (1 - df['acceptance_rate'])

In [ ]:

# Rank bucketing
def rank_bucket(score):
    if pd.isna(score):
        return 'Unknown'
    elif score <= -10:
        return 'Top 10'
    elif score <= -50:
        return 'Top 50'
    elif score <= -100:
        return 'Top 100'
    else:
        return 'Other'

In [ ]:
df['qs_rank_bucket'] = df['qs_rank_score'].apply(rank_bucket)

# Update feature list
features = [
    'undergrad_gpa', 'gre_quantitative_reasoning', 'gre_verbal_reasoning',
    'analytical_writing', 'acceptance_rate', 'qs_rank_score', 'qs_tier',
    'gpa_percentile', 'gre_avg', 'gpa_x_acceptancerate',
    'gpa_gre_interaction', 'gpa_acceptance_interaction',
    'profile_strength', 'qs_rank_bucket'
]

In [ ]:
# Create binary target: 1 if admitted, 0 otherwise
df['binary_decision'] = df['decision_grouped'].apply(lambda x: 1 if x == 1 else 0)

In [ ]:
X = df[features]
y = df['binary_decision']

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify numeric and categorical features
numeric_features = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

In [ ]:

# Preprocessing pipelines
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Define the pipeline
xgb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'))
])

In [ ]:
# Hyperparameter tuning
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [3, 6, 10],
    'classifier__learning_rate': [0.01, 0.1],
    'classifier__subsample': [0.8, 1],
    'classifier__colsample_bytree': [0.8, 1]
}

In [ ]:
grid_search = GridSearchCV(xgb_pipeline, param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [14:15:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [14:15:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [14:15:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [14:15:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packa

[CV] END classifier__colsample_bytree=0.8, classifier__learning_rate=0.01, classifier__max_depth=3, classifier__n_estimators=100, classifier__subsample=1; total time=   0.3s
[CV] END classifier__colsample_bytree=0.8, classifier__learning_rate=0.01, classifier__max_depth=3, classifier__n_estimators=100, classifier__subsample=1; total time=   0.4s
[CV] END classifier__colsample_bytree=0.8, classifier__learning_rate=0.01, classifier__max_depth=3, classifier__n_estimators=100, classifier__subsample=0.8; total time=   0.4s
[CV] END classifier__colsample_bytree=0.8, classifier__learning_rate=0.01, classifier__max_depth=3, classifier__n_estimators=100, classifier__subsample=1; total time=   0.4s
[CV] END classifier__colsample_bytree=0.8, classifier__learning_rate=0.01, classifier__max_depth=3, classifier__n_estimators=100, classifier__subsample=0.8; total time=   0.4s
[CV] END classifier__colsample_bytree=0.8, classifier__learning_rate=0.01, classifier__max_depth=3, classifier__n_estimators=1

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [14:15:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [14:15:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [14:15:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [14:15:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packa

[CV] END classifier__colsample_bytree=0.8, classifier__learning_rate=0.01, classifier__max_depth=3, classifier__n_estimators=200, classifier__subsample=0.8; total time=   0.6s
[CV] END classifier__colsample_bytree=0.8, classifier__learning_rate=0.01, classifier__max_depth=3, classifier__n_estimators=200, classifier__subsample=0.8; total time=   0.6s


...

[CV] END classifier__colsample_bytree=1, classifier__learning_rate=0.1, classifier__max_depth=10, classifier__n_estimators=100, classifier__subsample=1; total time=   1.2s
[CV] END classifier__colsample_bytree=1, classifier__learning_rate=0.1, classifier__max_depth=10, classifier__n_estimators=100, classifier__subsample=0.8; total time=   1.4s
[CV] END classifier__colsample_bytree=1, classifier__learning_rate=0.1, classifier__max_depth=10, classifier__n_estimators=100, classifier__subsample=1; total time=   1.3s
[CV] END classifier__colsample_bytree=1, classifier__learning_rate=0.1, classifier__max_depth=10, classifier__n_estimators=100, classifier__subsample=1; total time=   1.2s


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [14:15:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [14:15:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END classifier__colsample_bytree=1, classifier__learning_rate=0.1, classifier__max_depth=10, classifier__n_estimators=200, classifier__subsample=0.8; total time=   2.3s
[CV] END classifier__colsample_bytree=1, classifier__learning_rate=0.1, classifier__max_depth=10, classifier__n_estimators=200, classifier__subsample=1; total time=   1.9s
[CV] END classifier__colsample_bytree=1, classifier__learning_rate=0.1, classifier__max_depth=10, classifier__n_estimators=200, classifier__subsample=0.8; total time=   2.4s
[CV] END classifier__colsample_bytree=1, classifier__learning_rate=0.1, classifier__max_depth=10, classifier__n_estimators=200, classifier__subsample=0.8; total time=   2.2s
[CV] END classifier__colsample_bytree=1, classifier__learning_rate=0.1, classifier__max_depth=10, classifier__n_estimators=200, classifier__subsample=1; total time=   1.6s
[CV] END classifier__colsample_bytree=1, classifier__learning_rate=0.1, classifier__max_depth=10, classifier__n_estimators=200, classi

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [14:15:43] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['undergrad_gpa',
                                                                          'gre_quantitative_reasoning',
                                                                          'gre_verbal_reasoning',
                                                                          'analytical_writing',
                                                                          'acceptance_rate',
                                                                          'qs_rank_score',
                                                                          'qs_tier',
                                                                          'gpa_percentile',
                                                                          'gre_avg',
                                                                          'gpa_x_acceptanc...
                                                      monotone_constraints=None,
                                                      multi_strategy=None,
                                                      n_estimators=None,
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      random_state=42, ...))]),
             n_jobs=-1,
             param_grid={'classifier__colsample_bytree': [0.8, 1],
                         'classifier__learning_rate': [0.01, 0.1],
                         'classifier__max_depth': [3, 6, 10],
                         'classifier__n_estimators': [100, 200],
                         'classifier__subsample': [0.8, 1]},
             scoring='accuracy', verbose=2)

In [ ]:
# Best model evaluation
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

In [ ]:
print("Best Parameters:", grid_search.best_params_)
print(f'Accuracy: {accuracy:.4f}')
print(report)

Best Parameters: {'classifier__colsample_bytree': 1, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 100, 'classifier__subsample': 1}
Accuracy: 0.6620
              precision    recall  f1-score   support

           0       0.68      0.92      0.78     13030
           1       0.56      0.19      0.28      7095

    accuracy                           0.66     20125
   macro avg       0.62      0.55      0.53     20125
weighted avg       0.64      0.66      0.60     20125

